In [1]:
import numpy
import torch

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
X = torch.FloatTensor([0,0, 0,1, 1,0, 1,1]).reshape(-1,2).to(device)
Y = torch.FloatTensor([0,1,1,0]).reshape(-1,1).to(device)
print(X)
print(Y)

tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]], device='cuda:0')
tensor([[0.],
        [1.],
        [1.],
        [0.]], device='cuda:0')


In [4]:
# nn layers
linear1 = torch.nn.Linear(2,10,bias=True)
linear2 = torch.nn.Linear(10,10,bias=True)
linear3 = torch.nn.Linear(10,10,bias=True)
linear4 = torch.nn.Linear(10,1,bias=True)
sigmoid = torch.nn.Sigmoid()

# model
model = torch.nn.Sequential(linear1,sigmoid,linear2,sigmoid,
                            linear3,sigmoid,linear4,sigmoid).to(device)

In [5]:
# Define cost/loss & Optimizer
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=1)

In [6]:
digits = 0.1
MaxQ = 0
for step in range(10001):
    
    optimizer.zero_grad()
    hypothesis = model(X)
    
    # cost/loss function
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    
    quotient = int(step / digits)
    if(quotient >= 10):
        digits *= 10
    
    quotient = int(step / digits)
    if(MaxQ != quotient):
        print('Step {:6d} - log(Cost) {:.10f}'.format(
            step,numpy.log(cost.item())))
        MaxQ = quotient

Step      1 - log(Cost) -0.3664659677
Step      2 - log(Cost) -0.3664966656
Step      3 - log(Cost) -0.3664970096
Step      4 - log(Cost) -0.3664970096
Step      5 - log(Cost) -0.3664970956
Step      6 - log(Cost) -0.3664971815
Step      7 - log(Cost) -0.3664972675
Step      8 - log(Cost) -0.3664972675
Step      9 - log(Cost) -0.3664972675
Step     10 - log(Cost) -0.3664972675
Step     20 - log(Cost) -0.3664976975
Step     30 - log(Cost) -0.3664980414
Step     40 - log(Cost) -0.3664984714
Step     50 - log(Cost) -0.3664986434
Step     60 - log(Cost) -0.3664990733
Step     70 - log(Cost) -0.3664995033
Step     80 - log(Cost) -0.3664997612
Step     90 - log(Cost) -0.3665001052
Step    100 - log(Cost) -0.3665003632
Step    200 - log(Cost) -0.3665038028
Step    300 - log(Cost) -0.3665068125
Step    400 - log(Cost) -0.3665098222
Step    500 - log(Cost) -0.3665127459
Step    600 - log(Cost) -0.3665154976
Step    700 - log(Cost) -0.3665182493
Step    800 - log(Cost) -0.3665208291
Step    900 

In [7]:
# Accuracy computation
# True if hypothesis>0.5 else False
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(),
          '\nCorrect: ', predicted.detach().cpu().numpy(),
          '\nAccuracy: ', accuracy.item())


Hypothesis:  [[1.11791465e-04]
 [9.99828696e-01]
 [9.99842167e-01]
 [1.85544617e-04]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0
